In [46]:
import pymongo as pm
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt


In [47]:
from pymongo import MongoClient
import os as os
import pandas as pd
db_config= {
"MONGODB_ADDRESS"    : os.environ.get('KEY_MONGO_ADDRESS','mongodb-service.stag-data.svc.cluster.local'),
"MONGODB_PORT"       : os.environ.get('KEY_MONGO_PORT', '27017'),
"MONGODB_USERNAME"   : os.environ.get('KEY_MONGO_USER', 'IBSO-read-write'),
"MONGODB_PASSWORD"   : os.environ.get('KEY_MONGO_PASSWORD', 'NHWD5bw0xcMgy1Y'),
"MONGODB_DATABASE"   : os.environ.get('KEY_MONGO_DATABASE', 'admin')
}
mongo_client = MongoClient(
                ('mongodb://%s:%s@' + db_config["MONGODB_ADDRESS"] + ':' + db_config["MONGODB_PORT"] + '/' + db_config["MONGODB_DATABASE"]) % (
                    db_config["MONGODB_USERNAME"], db_config["MONGODB_PASSWORD"]), replicaset='rs0')
solutions_collection = mongo_client["main"]["solutions_parsed_version"]
solution_number = '2541754'
all_versions_list = list(solutions_collection.find({'Number': solution_number}).sort('ChangedOn_dt', -1))
all_versions_df = pd.DataFrame(all_versions_list)
latest_version = all_versions_list[0]


In [48]:
latest_version

{'_id': ObjectId('5f4e6203d1ae0500012d18de'),
 'AbapCode': [],
 'Attributes': [{'key': 'Responsible                                                                                         ',
   'Value': '8f41bb0beffaaf58d234743787e6cdc4459021bee0d02d7ab3b915610859d3a0'},
  {'key': 'Processor                                                                                           ',
   'Value': '8f41bb0beffaaf58d234743787e6cdc4459021bee0d02d7ab3b915610859d3a0'}],
 'Attributes_uh': [{'key': 'Responsible                                                                                         ',
   'Value': 'I306195'},
  {'key': 'Processor                                                                                           ',
   'Value': 'I306195'}],
 'Category': 'How To',
 'ChangedOn': '01.09.2020',
 'ChangedOn_dt': datetime.datetime(2020, 9, 1, 0, 0),
 'CorrectionInstructionsObjectList': [],
 'CreatedOn': '28.09.2017',
 'CreatedOn_dt': datetime.datetime(2017, 9, 28, 0, 0),
 'Current

In [49]:
df_all=pd.DataFrame(list(solutions_collection.find({}).limit(1)))
df_all.columns

Index(['_id', 'AbapCode', 'Attributes', 'Attributes_uh', 'Category',
       'ChangedOn', 'ChangedOn_dt', 'CorrectionInstructionsObjectList',
       'CreatedOn', 'CreatedOn_dt', 'CurrentStatus', 'ExpirationDate',
       'HasPreconditions', 'InternalMemo', 'Languages', 'Number',
       'OtherSAPComponents', 'Patches', 'PrimarySAPComponent', 'Priority',
       'Product', 'RatingInfo', 'RefNumber', 'ReferenceTo', 'ReferencedBy',
       'ReleasedOn', 'ReleasedOn_dt', 'SoftwareComponents', 'SupportPackages',
       'TargetStatus', 'TextSections', 'Title', 'TranslationFlag', 'Type',
       'Version', 'extracted_at', 'extracted_at_dt', 'sha_hashed', 'Symptom',
       'Solution', 'Reason&Prerequisites', 'KeyTerms', 'ReproducingTheIssue',
       'Cause', 'AbapCode_parsed', 'NormalizedTitle', 'NormalizedSymptom',
       'NormalizedSolution', 'NormalizedReason&Prerequisites',
       'NormalizedKeyTerms', 'NormalizedReproducingTheIssue',
       'NormalizedCause', 'NormalizedText', 'TermCount', 'par

In [50]:
solution_data = pd.DataFrame(list(solutions_collection.find({},{'_id':0,'Number':1,'Version':1, 'Category':1, 'ChangedOn_dt':1, 
                                                                'CreatedOn_dt':1, 'PrimarySAPComponent':1, 'OtherSAPComponents':1,
                                                                'ReleasedOn_dt':1,  'Priority':1,
                                                                'Product':1, 'Type':1,  'Title':1,
                                                                'Symptom':1, 'Solution':1, 'KeyTerms':1,'TermCount':1
                                                               })))

In [51]:
solution_data.shape

(2676523, 16)

In [12]:
# solution_data_grouped - []
# solution_data_grouped = solution_data.groupby('Number').max('Version')
# A Solution can have multiple versions - get the latest version of each Solution

idx = solution_data.groupby(['Number'])['Version'].transform(max) == solution_data['Version']

solution_data_grouped = solution_data[idx]

In [22]:
solution_data_grouped.shape
list(solution_data_grouped.columns)

['Category',
 'ChangedOn_dt',
 'CreatedOn_dt',
 'Number',
 'OtherSAPComponents',
 'PrimarySAPComponent',
 'Priority',
 'Product',
 'ReleasedOn_dt',
 'Title',
 'Type',
 'Version',
 'Symptom',
 'Solution',
 'KeyTerms',
 'TermCount']

In [23]:
def get_solution_component(solutionNumber, solution_data_grouped):
    return solution_data_grouped.loc[solution_data_grouped['Number'] == solutionNumber, 'PrimarySAPComponent']

    

In [54]:
dum = get_solution_component('15451', solution_data_grouped)

In [57]:
type(dum)

pandas.core.series.Series

In [59]:
dum.values[0]

'fi-aa-aa-f'

In [ ]:
get_solution_component('455942', solution_data_grouped)[0]

In [26]:
def get_solution_otherSAPcomponents(solutionNumber, solution_data_grouped):
    return solution_data_grouped.loc[solution_data_grouped['Number'] == solutionNumber, 'OtherSAPComponents']


In [28]:
get_solution_otherSAPcomponents('455942', solution_data_grouped)

1390699    []
Name: OtherSAPComponents, dtype: object

In [60]:
AIT_Customer_Solution_Validation = pd.read_csv('AIT_UC2_Customer_Solution_Matching.csv', encoding='latin-1')


In [61]:
AIT_Customer_Solution_Validation.head(10)

,Customer Number,Installation Number,System No,Solution Number,Solution Title,Solution App Component,Is the Solution Relevant?,Top-k Relevant Solutions (proposed & missed) sorted by relevancy,Comment
0,490533,20848565,NaN,455942,455942 - Log analysis: Termination TSV_TNEW_PA...,BC-CUS-TOL-ALO,NaN,NaN,NaN
1,490533,20848565,NaN,2585851,2585851 - Automatically created customer incid...,BC-FES-GUI,NaN,NaN,NaN
2,490533,20848565,NaN,3154651,3154651 - RKKBCAL2: Runtime error BCD_FIELD_OV...,CO-PC-PCP,NaN,NaN,NaN
3,530444,20848565,NaN,0,NaN,NaN,NaN,NaN,NaN
4,530444,20848565,NaN,3148891,3148891 - ITAB_DUPLICATE_KEY dump when updatin...,TM-MD-TN-LOC,NaN,NaN,NaN
5,530444,20848565,NaN,3151018,3151018 - Dump in the method redirect_navigati...,CA-WUI-UI-RT,NaN,NaN,NaN
6,530444,20848565,NaN,2842983,2842983 - BOE Multiple due dates in Manage Aut...,FI-FIO-AP,NaN,NaN,NaN
7,530444,20848565,NaN,3139451,3139451 - SAP GUI for HTML: Tree columns are d...,BC-FES-ITS,NaN,NaN,NaN
8,530444,20848565,NaN,2844993,2844993 - BOE Due Date for Customer Incoming P...,FI-FIO-AR,NaN,NaN,NaN
9,530444,20848565,NaN,3156586,3156586 - Balance Sheet Valuation Can't Update...,MM-IM-VP,NaN,NaN,NaN


In [62]:
list(AIT_Customer_Solution_Validation.columns)

['Customer Number',
 'Installation Number',
 'System No',
 'Solution Number',
 'Solution Title',
 'Solution App Component',
 'Is the Solution Relevant?',
 'Top-k Relevant Solutions (proposed & missed) sorted by relevancy',
 'Comment']

In [70]:
for x in AIT_Customer_Solution_Validation['Solution Number']:
    print('Solution number is ', x)
    dum = get_solution_component(str(x), solution_data_grouped)
    print('The component for this Solution is ', dum)

Solution number is  455942
The component for this Solution is  1390699    bc-cus-tol-alo
Name: PrimarySAPComponent, dtype: object
Solution number is  2585851
The component for this Solution is  1891588    bc-fes-gui
Name: PrimarySAPComponent, dtype: object
Solution number is  3154651
The component for this Solution is  Series([], Name: PrimarySAPComponent, dtype: object)
Solution number is  0
The component for this Solution is  Series([], Name: PrimarySAPComponent, dtype: object)
Solution number is  3148891
The component for this Solution is  Series([], Name: PrimarySAPComponent, dtype: object)
Solution number is  3151018
The component for this Solution is  Series([], Name: PrimarySAPComponent, dtype: object)
Solution number is  2842983
The component for this Solution is  2198155    fi-fio-ap
2199063    fi-fio-ap
Name: PrimarySAPComponent, dtype: object
Solution number is  3139451
The component for this Solution is  2627593    bc-fes-its
2627863    bc-fes-its
Name: PrimarySAPComponent,

KeyboardInterrupt: 

In [64]:
AIT_Customer_Solution_Validation['PrimarySAPComponent'] = [get_solution_component(x, solution_data_grouped) for x in AIT_Customer_Solution_Validation['Solution Number']]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [45]:
AIT_Customer_Solution_Validation.head(10)

,Customer Number,Installation Number,System No,Solution Number,Solution Title,Solution App Component,Is the Solution Relevant?,Top-k Relevant Solutions (proposed & missed) sorted by relevancy,Comment,PrimarySAPComponent
0,490533,20848565,NaN,455942,455942 - Log analysis: Termination TSV_TNEW_PA...,BC-CUS-TOL-ALO,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
1,490533,20848565,NaN,2585851,2585851 - Automatically created customer incid...,BC-FES-GUI,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
2,490533,20848565,NaN,3154651,3154651 - RKKBCAL2: Runtime error BCD_FIELD_OV...,CO-PC-PCP,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
3,530444,20848565,NaN,0,NaN,NaN,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
4,530444,20848565,NaN,3148891,3148891 - ITAB_DUPLICATE_KEY dump when updatin...,TM-MD-TN-LOC,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
5,530444,20848565,NaN,3151018,3151018 - Dump in the method redirect_navigati...,CA-WUI-UI-RT,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
6,530444,20848565,NaN,2842983,2842983 - BOE Multiple due dates in Manage Aut...,FI-FIO-AP,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
7,530444,20848565,NaN,3139451,3139451 - SAP GUI for HTML: Tree columns are d...,BC-FES-ITS,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
8,530444,20848565,NaN,2844993,2844993 - BOE Due Date for Customer Incoming P...,FI-FIO-AR,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."
9,530444,20848565,NaN,3156586,3156586 - Balance Sheet Valuation Can't Update...,MM-IM-VP,NaN,NaN,NaN,"Series([], Name: PrimarySAPComponent, dtype: o..."


In [41]:
from pymongo import MongoClient
db_config= {
"MONGODB_ADDRESS"    : os.environ.get('KEY_MONGO_ADDRESS','mongodb-service.stag-data.svc.cluster.local'),
"MONGODB_PORT"       : os.environ.get('KEY_MONGO_PORT', '27017'),
"MONGODB_USERNAME"   : os.environ.get('KEY_MONGO_USER', 'IBSO-read-write'),
"MONGODB_PASSWORD"   : os.environ.get('KEY_MONGO_PASSWORD', 'NHWD5bw0xcMgy1Y'),
"MONGODB_DATABASE"   : os.environ.get('KEY_MONGO_DATABASE', 'admin')
}
mongo_client = MongoClient(
                ('mongodb://%s:%s@' + db_config["MONGODB_ADDRESS"] + ':' + db_config["MONGODB_PORT"] + '/' + db_config["MONGODB_DATABASE"]) % (
                    db_config["MONGODB_USERNAME"], db_config["MONGODB_PASSWORD"]), replicaset='rs0')
infodocs_metadata = mongo_client["IBSO"]["infodocs_metadata"]

In [42]:
df_infodocs_metadata = pd.DataFrame(infodocs_metadata.find({"incident_created_at_dt":{
   "$gte": datetime.datetime(2022,2,1),
    "$lt": datetime.datetime(2022,4,1)
}}))

In [43]:
df_infodocs_metadata.head(10)

,_id,css_object_id,incident_created_at_dt,customer_name,customer_no,component,description,prod_version,communication_logs,solns,description_problem_description,Reply,installation_no,system_no
0,62681b684cc370bff2568330,002024211700000044652022,2022-02-01 00:01:19,VISTRA HOLDINGS (UK) LIMITED,0002054244,XX-S4C-OPR-SRV,Foreign Key Errors Present in Q2P Project,SAP S/4HANA CLOUD 2111,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...","[10, 5]",Foreign Key Errors Present in Q2P Project. For...,"see * in referenced incident, no clue what's g...",0020848565,741368080
1,62681b684cc370bff25696ef,002075129400000737852022,2022-02-01 00:04:31,Distribuidora de Electricidad del S,0000875428,MOB-APP-MAO,Error transmitting meter movement from Work Ma...,SAP MOBILE PLATFORM 3.0,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...",[0002504590],Error transmitting meter movement from Work Ma...,,0020853665,900062266
2,62681b684cc370bff25696e0,002075129400000737372022,2022-02-01 00:04:31,Jemena Limited,0000237010,LOD-SF-EC-FOO,SM - Unable to maintain translation for Founda...,SUCCESSFACTORS BIZX CORE 2111,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...","[0002197595, 0002079466]",SM - Unable to maintain translation for Founda...,,0090276236,740254631
3,62681b684cc370bff2568331,002024211700000044662022,2022-02-01 00:06:16,Mundipharma IT Services Limited,0001572344,XX-HST-OPR,PS4MUN - System is down,SAP S/4HANA 1909,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...",[0000000010],PS4MUN - System is down. Dear * is to inform y...,Dear * Taken / Analysis]&nbsp;System availabil...,0021196753,500201332
4,62681b684cc370bff2568337,002024211700000044772022,2022-02-01 00:06:21,Mann+Hummel GmbH,0000011597,FI-CNT-GL,Foreign Key Errors Present in Q2P Project,SAP S/4HANA CLOUD 2111,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...",[10],Foreign Key Errors Present in Q2P Project. For...,Exemption granted,0020848565,741190318
5,62681b684cc370bff256833b,002024211700000044852022,2022-02-01 00:08:58,SAP Cloud,0001119571,SCM-IBP-OPS-SIZ,OOM events per week,SAP IBP OD 2111,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...",[10],OOM events per week. OOM * per week&nbsp;&nbsp...,This ticket has been passed to SAP DSC IBP SRE...,0020848565,741033410
6,62681b684cc370bff25696f1,002075129400000737962022,2022-02-01 00:11:29,Luminous Power Technologies Pvt Ltd,0001055712,XX-HST-OPR-INC,"Getting error code 9603 , communication link f...",SAP TDMS 4.0,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...",[0000000010],"Getting error code 9603 , communication link f...",Dear * Taken / Analysis]We are working on the ...,0021247250,500246845
7,62681b684cc370bff256b04f,002075129500000820532022,2022-02-01 00:14:24,Southern California Edison,0000718736,HAN-DB-PERF,SAP GRC EAM on HANA DB - System slowness due t...,NW AS ABAP 7.52,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...","[0002000000, 0001969700, 0001999993, 000218016...",SAP GRC EAM on HANA DB - System slowness due t...,,0020240597,311693095
8,62681b684cc370bff25696f0,002075129400000737862022,2022-02-01 00:15:29,Department of Customer Service,0000691023,FI-TV-COS,Credit Card receipts getting doubled,SAP S/4HANA 2020,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...","[0001239839, 0001523821]",Credit Card receipts getting doubled. Product ...,,0021216040,800643326
9,62681b684cc370bff25696f4,002075129400000738012022,2022-02-01 00:15:29,Empresa Colombiana De Petroleos S.A,0000671855,LOD-SF-LMS-INT,Administrators LMS access error,SUCCESSFACTORS ONBOARDING 2111,"[{'datetime': '', 'text': '<p>', 'type': ''}, ...",[0002264167],Administrators LMS access error. Product Area ...,,0090037162,740905075


In [71]:
df_infodocs_metadata.shape

(101294, 14)

In [72]:
df_infodocs_metadata.columns

Index(['_id', 'css_object_id', 'incident_created_at_dt', 'customer_name',
       'customer_no', 'component', 'description', 'prod_version',
       'communication_logs', 'solns', 'description_problem_description',
       'Reply', 'installation_no', 'system_no'],
      dtype='object')

In [73]:
from pymongo import MongoClient
db_config= {
"MONGODB_ADDRESS"    : os.environ.get('KEY_MONGO_ADDRESS','mongodb-service.stag-data.svc.cluster.local'),
"MONGODB_PORT"       : os.environ.get('KEY_MONGO_PORT', '27017'),
"MONGODB_USERNAME"   : os.environ.get('KEY_MONGO_USER', 'IBSO-read-write'),
"MONGODB_PASSWORD"   : os.environ.get('KEY_MONGO_PASSWORD', 'NHWD5bw0xcMgy1Y'),
"MONGODB_DATABASE"   : os.environ.get('KEY_MONGO_DATABASE', 'admin')
}
mongo_client = MongoClient(
                ('mongodb://%s:%s@' + db_config["MONGODB_ADDRESS"] + ':' + db_config["MONGODB_PORT"] + '/' + db_config["MONGODB_DATABASE"]) % (
                    db_config["MONGODB_USERNAME"], db_config["MONGODB_PASSWORD"]), replicaset='rs0')
infodocs_parsed_version = mongo_client["IBSO"]["infodocs_parsed_version"]

In [ ]:
df_infodocs_parsed_version = pd.DataFrame(infodocs_parsed_version.find({"incident_created_at_dt":{
   "$gte": datetime.datetime(2010,2,1),
    "$lt": datetime.datetime(2022,4,1)
}}))

In [75]:
df_infodocs_parsed_version.shape

(0, 0)

In [76]:
def connect_mongo():
    try:
        mongo_client = MongoClient(
            ('mongodb://%s:%s@' + MONGODB_ADDRESS + ':' + MONGODB_PORT + '/' + MONGODB_DATABASE) % (
                MONGODB_USERNAME, MONGODB_PASSWORD),
            replicaset='rs0')
        return mongo_client
    except:
        print("connection wrong")
        exit()

In [77]:
# Set up db connection
mongo_client = connect_mongo()

connection wrong


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
